<div align='center' ><font size='6'>《计算机视觉与机器学习》课外实践练习三</font></div> 

队长学号：20211000581

队长姓名：林玉雯

学号：20211004187

姓名：王曼曼

学号：20211000742

姓名：张玥

## 实习要求：

使用pytorch框架的模块完成任务：


**一、使用提供的三种卷积网络形式对HelloRS32数据集分类**
  1. model1：编写并实现简单的双层卷积神经网络的训练及预测，精度达到**86%**以上
  
  
  2. model2：编写并实现三层卷积神经网络的训练及预测，精度达到**88%**以上
      * 使用了relu作为激活函数
      * 在线性层中加入dropout
      * 使用局部响应归一化层lrn
      * 相比与modle1拥有更深的卷积层
      
      
  3. model3：编写并实现很多层的卷积神经网络的训练及预测，精度达到**91%**以上
      * 更小的卷积核尺寸、卷积步长
      * 使用批量归一化batchnorm代替局部响应归一化层lrn
      * 更深的网络
  
  
   4. **选做**：尝试在model3的基础上加入skip connection，进一步提升精度

**二、完成神经网络网络在cifar-10上的训练、预测函数**

# 使用pytorch框架的模块完成任务

Dropout、Batch Norm和2D卷积是计算机视觉中深度学习的主要工具。

在本作业的最后一部分，放弃之前的代码库，转而迁移到流行的深度学习框架之一: PyTorch。

## PyTorch是什么?

PyTorch是一个在Tensor对象上执行动态计算图形的系统，其行为类似于numpy ndarray。它带有一个强大的自动区分引擎，消除了手动反向传播的需要。

### 为什么采用Pytorch

* 我们的代码现在将在gpu上运行速度更快的训练。当使用像PyTorch或TensorFlow这样的框架时，你可以为你自己的自定义神经网络架构利用GPU的能力，而不必直接编写CUDA代码(这超出了本类的范围)。
* 我们希望你准备好在你的项目中使用这些框架之一，这样你就可以更有效地进行试验，而不是手工编写你想要使用的所有功能。
* 我们要你站在巨人的肩膀上！PyTorch是很棒的框架，会让你的生活变得更简单，现在你已经理解了它的本质，你可以自由地使用它了:)
* 我们希望你能接触到你在学术界或工业界可能会遇到的那种深度学习代码。

### PyTorch版本

假设你正在使用**PyTorch 1.0版本**。在之前的一些版本中(比如0.4之前)，Tensor必须被包装在变量对象中才能在autograd中使用；然而变量现在已被弃用。此外，1.0还将Tensor的数据类型从它的设备中分离出来，并使用numpy风格的工厂来构造Tensor，而不是直接调用Tensor构造函数。

## 我将如何学习PyTorch?

[PyTorch 向导](https://github.com/jcjohnson/pytorch-examples) 可以参考github的这些样例。

也可以看Pytorch详细的 [API文档](http://pytorch.org/docs/stable/index.html)。如果你有其他API文档没有解决的问题, [PyTorch forum](https://discuss.pytorch.org/) 也是一个可以参考的地方。


### PyTorch可以分为**三个不同的抽象层次**进行构建：
1. Barebone PyTorch：**抽象级别1**，直接使用最低级别的PyTorch张量。
2. PyTorch模块API：**抽象级别2**，使用`nn.Module`定义任意神经网络结构。
3. PyTorch顺序API：**抽象级别3**，使用`nn.Sequential`可以非常方便地定义线性前馈网络。


以下是一个比较表：

| API           | 灵活性 | 方便性 |
|---------------|-------------|-------------|
| Barebone      |高        | 低         |
| `nn.Module`     | 高       | 中      |
| `nn.Sequential` | 低         | 高        |

# Part I. 准备阶段

首先，加载HelloRS32数据集，对其进行预处理，并以mini_batch遍历它；PyTorch为我们提供了方便的工具来自动化这个过程。

## 导入模块

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import ImageFolder

## 超参数设置

In [2]:
batch_size = 4  
lr = 0.001 
epoch = 40
print_every = 1000  # 输出损失的频率

## HelloRS32数据集加载

In [3]:
train_path = "D:/CVML/RS32/train"
val_path = "D:/CVML/RS32/val"
test_path = "D:/CVML/RS32/test"
out_dir = "./model"

data_transform = {
    "train": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0, 0, 0], [1, 1, 1])]),
    "test": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0, 0, 0], [1, 1, 1])])}

trainset = ImageFolder(train_path, transform=data_transform["train"])
valset = ImageFolder(val_path, transform=data_transform["test"])
testset = ImageFolder(test_path, transform=data_transform["test"])

train_loader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,
                                            shuffle=True,num_workers=4)
val_loader = torch.utils.data.DataLoader(valset,batch_size=batch_size, 
                                         shuffle=True,num_workers=4)
test_loader = torch.utils.data.DataLoader(testset,batch_size=batch_size,
                                          shuffle=True,num_workers=4)

你可以**通过设置下面的flag为True使用GPU**。本课程的任务可以不需要使用GPU，但使用GPU可以加速计算。请注意，如果你的计算机没有启用CUDA，`torch.cuda.is_available()`将返回False，将回退到CPU模式。
全局变量`dtype `和`device`将控制整个赋值过程中的数据类型。

In [4]:
dtype = torch.float32  #使用float作为数据类型

USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


# Part II. 模型的设置
新建 `model(*).py`,参照给定的网络构造实现不同卷积神经网络对遥感数据集HelloRS32进行分类：

你需要查看 `torch.nn` 和 `torch.nn.functional` 来了解pytorch中卷积、池化、激活函数、全连接等功能是如何实现的。

当你成功实现model之后，你可以尝试使用更多网络，如`resnet`、`vggnet`、`mobilenet`、`efficientnet`等更强大的网络来在HelloRS32上训练，以提高你的精度，当然这也意味着需要更大的算力

In [59]:
import train
from models.model1 import Mode11
from models.model2 import Model2
from models.model3 import Model3
from models.model4 import Model4

net = Model4()
net = net.to(device)

# Part III. PyTorch的训练

在开始网络的训练之前，我们还需要设置网络的损失函数、优化器，这些设置对于网络的训练也具有重要的影响，但本课程暂不要求对这些设置具有深刻的理解，我们会给出一些示例，你在后续的训练中可以更换不同的损失函数、优化器以及`scheduler`，以提高训练的精度和效率。

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

完成前期的设置之后，我们就可以开始编写训练的代码了，打开`train.py`，完成`train`和`test`函数，并在下面运行训练和测试的代码。

In [14]:
import time 
tic = time.time()
train.train_val(train_loader, val_loader, net, criterion, optimizer, epoch, print_every, device, scheduler,
            output_dir=out_dir)

toc = time.time()
print(toc - tic)

Epoch: 0
training Loss: 1.833|Acc: 31.081%(2396/7709)
val Loss: 1.583 | Acc: 39.155%
Saving..
Epoch: 1
training Loss: 1.194|Acc: 43.670%(6733/15418)
val Loss: 0.950 | Acc: 52.552%
Saving..
Epoch: 2
training Loss: 0.904|Acc: 51.252%(11853/23127)
val Loss: 0.886 | Acc: 57.443%
Saving..
Epoch: 3
training Loss: 0.796|Acc: 55.983%(17263/30836)
val Loss: 0.694 | Acc: 61.802%
Saving..
Epoch: 4
training Loss: 0.714|Acc: 59.414%(22901/38545)
val Loss: 0.629 | Acc: 64.609%
Saving..
Epoch: 5
training Loss: 0.645|Acc: 62.064%(28707/46254)
val Loss: 0.593 | Acc: 66.853%
Saving..
Epoch: 6
training Loss: 0.579|Acc: 64.314%(34706/53963)
val Loss: 0.718 | Acc: 67.464%
Saving..
Epoch: 7
training Loss: 0.546|Acc: 66.140%(40790/61672)
val Loss: 1.203 | Acc: 66.956%
Epoch: 8
training Loss: 0.511|Acc: 67.702%(46972/69381)
val Loss: 0.508 | Acc: 68.483%
Saving..
Epoch: 9
training Loss: 0.481|Acc: 69.128%(53291/77090)
val Loss: 0.641 | Acc: 69.418%
Saving..
Epoch: 10
training Loss: 0.454|Acc: 70.337%(59645/84

## 测试集

我们通过训练得到了精度最高的模型之后，在测试集上运行我们的模型，并得到测试精度。

In [7]:
#加载模型参数
#model1
ckp = "D:/CVML/lab3/Quiz/model/epoch039.pth"
pretrained_dict = torch.load(ckp)
net.load_state_dict(pretrained_dict["net"])

#测试
train.predict(test_loader, net, criterion, device)

testing Loss: 0.451 | Acc: 87.259%


In [10]:
#加载模型参数
#model2
ckp = "D:/CVML/lab3/Quiz/model2/epoch039.pth"
pretrained_dict = torch.load(ckp)
net.load_state_dict(pretrained_dict["net"])

#测试
train.predict(test_loader, net, criterion, device)

testing Loss: 0.459 | Acc: 90.516%


In [57]:
#加载模型参数
#model3
ckp = "D:/CVML/lab3/Quiz/model3/epoch039.pth"
pretrained_dict = torch.load(ckp)
net.load_state_dict(pretrained_dict["net"])

#测试
train.predict(test_loader, net, criterion, device)

testing Loss: 0.376 | Acc: 91.532%


In [73]:
#加载模型参数
#model4
ckp = "D:/CVML/lab3/Quiz/model4/epoch039.pth"
pretrained_dict = torch.load(ckp)
net.load_state_dict(pretrained_dict["net"])

#测试
train.predict(test_loader, net, criterion, device)

testing Loss: 0.401 | Acc: 93.278%
